In [16]:
import logging
from kiteconnect import KiteConnect
from dotenv import load_dotenv
import os
import json

logging.basicConfig(level=logging.DEBUG)

# Load .env file
load_dotenv("config/.env")

with open("config/zerodha.json", "r") as file:
    config = json.load(file)

class Login:
    def __init__(self):
        self.api_key = os.getenv("api_key")
        self.api_secret = os.getenv("api_secret")
        self.access_token = config['access_token']
        self.kite = KiteConnect(api_key=self.api_key)
        self.kite.set_access_token(self.access_token)
        
        # try to see if token is still valid
        try:
            profile_data = self.kite.profile()
            logging.info(f"User ID: {profile_data['user_id']}, {profile_data['user_name']} Connected!")
        except Exception as e:
            logging.error(f"{e}, Getting new token...")
            self.get_new_token()
        
    def get_new_token(self):
        url = self.kite.login_url()
        print(url)
        request_token = input("Enter Request token: ")
        new_session_data = self.kite.generate_session(request_token=request_token, api_secret=self.api_secret)
        logging.info(f"User ID: {new_session_data['user_id']}, {new_session_data['user_name']} Connected!")
        self.access_token = new_session_data["access_token"]
        self.kite.set_access_token(self.access_token)
        config_data = {
            'access_token' : self.access_token
        }
        with open('config/zerodha.json', 'w') as jsonfile:
            json.dump(config_data, jsonfile, indent=4)
    
    def get_broker_handler(self):
        return self.kite


In [4]:
import logging
from kiteconnect import KiteTicker
import os
from dotenv import load_dotenv
import json
import time
logging.basicConfig(level=logging.DEBUG)

# Load .env file
load_dotenv("config/.env")

with open("config/zerodha.json", "r") as file:
    config = json.load(file)
    
api_key = os.getenv("api_key")
access_token = config['access_token']

# Initialise
kws = KiteTicker(api_key, access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    logging.debug("Ticks: {}".format(ticks))

def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([738561])

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_LTP, [738561])

def on_close(ws, code, reason):
    # On connection close stop the event loop.
    # Reconnection will not happen after executing `ws.stop()`
    print("Connect is closing......")
    ws.stop()

def on_order_update(ws, data):
    # On connection close stop the event loop.
    # Reconnection will not happen after executing `ws.stop()`
    logging.info(f"orderUpdate : {data}")

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close
kws.on_order_update = on_order_update

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
try:
    kws.connect(threaded=True)
except KeyboardInterrupt:
    logging.info("WebSocket connection interrupted by user.")
    kws.stop()

DEBUG:root:Ticks: [{'tradable': True, 'mode': 'ltp', 'instrument_token': 738561, 'last_price': 1231.0}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'ltp', 'instrument_token': 738561, 'last_price': 1231.05}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'ltp', 'instrument_token': 738561, 'last_price': 1231.0}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'ltp', 'instrument_token': 738561, 'last_price': 1230.9}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'ltp', 'instrument_token': 738561, 'last_price': 1230.6}]
INFO:root:orderUpdate : {'account_id': 'BI3034', 'unfilled_quantity': 0, 'checksum': '', 'placed_by': 'BI3034', 'order_id': '241219201576779', 'exchange_order_id': '1100000043979126', 'parent_order_id': None, 'status': 'OPEN', 'status_message': None, 'status_message_raw': None, 'order_timestamp': '2024-12-19 13:48:34', 'exchange_update_timestamp': '2024-12-19 13:48:34', 'exchange_timestamp': '2024-12-19 13:48:34', 'variety': 'regular', 'exchange': 'NSE', 'tradingsymbol': 'IT

In [5]:
kws.unsubscribe([738561])

True

In [7]:
kws.set_mode(kws.MODE_LTP, [738561])

True

In [17]:
from pprint import pprint

In [18]:
obj = Login()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/11" 200 None
INFO:root:User ID: BI3034, Lucky Kushwaha Connected!


In [19]:
kite_handler = obj.get_broker_handler()

In [20]:
data = kite_handler.positions()

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/11" 200 None


In [30]:
kite_handler.instruments('NFO')

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/11" 200 516711


[{'instrument_token': 8961282,
  'exchange_token': '35005',
  'tradingsymbol': 'NIFTY24DECFUT',
  'name': 'NIFTY',
  'last_price': 0.0,
  'expiry': datetime.date(2024, 12, 26),
  'strike': 0.0,
  'tick_size': 0.05,
  'lot_size': 25,
  'instrument_type': 'FUT',
  'segment': 'NFO-FUT',
  'exchange': 'NFO'},
 {'instrument_token': 8961538,
  'exchange_token': '35006',
  'tradingsymbol': 'NIFTY25JANFUT',
  'name': 'NIFTY',
  'last_price': 0.0,
  'expiry': datetime.date(2025, 1, 30),
  'strike': 0.0,
  'tick_size': 0.05,
  'lot_size': 25,
  'instrument_type': 'FUT',
  'segment': 'NFO-FUT',
  'exchange': 'NFO'},
 {'instrument_token': 8963330,
  'exchange_token': '35013',
  'tradingsymbol': 'NIFTY25FEBFUT',
  'name': 'NIFTY',
  'last_price': 0.0,
  'expiry': datetime.date(2025, 2, 27),
  'strike': 0.0,
  'tick_size': 0.05,
  'lot_size': 75,
  'instrument_type': 'FUT',
  'segment': 'NFO-FUT',
  'exchange': 'NFO'},
 {'instrument_token': 8966402,
  'exchange_token': '35025',
  'tradingsymbol': 'B

# Instrument


In [11]:
def fetchInstrumentsFromServer():
    instrumentsList = []
    try:
      brokerHandle = kite_handler
      logging.info('Going to fetch instruments from server...')
      instrumentsList = brokerHandle.instruments('NSE')
      instrumentsListFnO = brokerHandle.instruments('NFO')
      # Add FnO instrument list to the main list
      instrumentsList.extend(instrumentsListFnO)
      logging.info('Fetched %d instruments from server.', len(instrumentsList))
    except Exception as e:
      logging.exception("Exception while fetching instruments from server")
    return instrumentsList

def getInstrumentDataBySymbol(tradingSymbol):
    return Instruments.symbolToInstrumentMap[tradingSymbol]

def getInstrumentDataByToken(instrumentToken):
    return Instruments.tokenToInstrumentMap[instrumentToken]

In [9]:
Instruments = fetchInstrumentsFromServer()
# Instruments

INFO:root:Going to fetch instruments from server...
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NSE HTTP/11" 200 126878
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/11" 200 502858
INFO:root:Fetched 44724 instruments from server.


In [17]:
# Creating the map
symbol_to_token = {item['tradingsymbol']: item['instrument_token'] for item in Instruments}
token_to_symbol = {item['instrument_token']: item['tradingsymbol'] for item in Instruments}

In [23]:
def get_symbol_to_token(tradingsymbol):
    return symbol_to_token.get(tradingsymbol)
def get_token_to_symbol(instrument_token):
    return token_to_symbol.get(instrument_token)

In [25]:
# Example usage
tradingsymbol = 'NIFTY 50'
get_symbol_to_token(tradingsymbol)

256265

In [26]:
instrument_token = 256265
get_token_to_symbol(instrument_token)

'NIFTY 50'

# Order Placement

In [10]:
class KiteConstants:
    #Products
    PRODUCT_MIS = "MIS"
    PRODUCT_CNC = "CNC"
    PRODUCT_NRML = "NRML"
    PRODUCT_CO = "CO"

    # Order types
    ORDER_TYPE_MARKET = "MARKET"
    ORDER_TYPE_LIMIT = "LIMIT"
    ORDER_TYPE_SLM = "SL-M"
    ORDER_TYPE_SL = "SL"

    # Varities
    VARIETY_REGULAR = "regular"
    VARIETY_CO = "co"
    VARIETY_AMO = "amo"
    VARIETY_ICEBERG = "iceberg"
    VARIETY_AUCTION = "auction"

    # Transaction type
    TRANSACTION_TYPE_BUY = "BUY"
    TRANSACTION_TYPE_SELL = "SELL"

    # Validity
    VALIDITY_DAY = "DAY"
    VALIDITY_IOC = "IOC"
    VALIDITY_TTL = "TTL"

    # Position Type
    POSITION_TYPE_DAY = "day"
    POSITION_TYPE_OVERNIGHT = "overnight"

    # Exchanges
    EXCHANGE_NSE = "NSE"
    EXCHANGE_BSE = "BSE"
    EXCHANGE_NFO = "NFO"
    EXCHANGE_CDS = "CDS"
    EXCHANGE_BFO = "BFO"
    EXCHANGE_MCX = "MCX"
    EXCHANGE_BCD = "BCD"

    # Margins segments
    MARGIN_EQUITY = "equity"
    MARGIN_COMMODITY = "commodity"

    # Status constants
    STATUS_COMPLETE = "COMPLETE"
    STATUS_REJECTED = "REJECTED"
    STATUS_CANCELLED = "CANCELLED"

    # GTT order type
    GTT_TYPE_OCO = "two-leg"
    GTT_TYPE_SINGLE = "single"

    # GTT order status
    GTT_STATUS_ACTIVE = "active"
    GTT_STATUS_TRIGGERED = "triggered"
    GTT_STATUS_DISABLED = "disabled"
    GTT_STATUS_EXPIRED = "expired"
    GTT_STATUS_CANCELLED = "cancelled"
    GTT_STATUS_REJECTED = "rejected"
    GTT_STATUS_DELETED = "deleted"


In [11]:
class OrderState:
    OPEN = "OPEN"
    COMPLETE = "COMPLETE"
    REJECTED = "REJECTED"
    CANCELLED = "CANCELLED"

In [12]:
class Order:
    def __init__(self, variety, exchange, trading_symbol, transaction_type, quantity, product, order_type, price=None, trigger_price=None):
        self.variety=variety
        self.exchange=exchange
        self.trading_symbol=trading_symbol
        self.transaction_type=transaction_type
        self.quantity=quantity
        self.product=product
        self.order_type=order_type
        self.price=price
        self.trigger_price=trigger_price
        self.average_price=None # Average price at which the order is filled
        self.order_id=None
        self.order_status=None
        self.filled_qty = 0 # Filled quantity
        self.pending_qty = 0 # Pending qty
        self.order_place_timestamp = None # Timestamp when the order is placed
        self.order_timestamp = None # Broker timestamp
        self.exchange_timestamp = None #Exchange timestamp
        self.lastOrderUpdateTimestamp = None # Applicable if you modify the order Ex: Trailing SL
        self.message = None # In case any order rejection or any other error save the response from broker in this field
    
    def set_average_price(self, average_price):
        self.average_price = average_price
    
    def set_order_id(self, order_id):
        self.order_id = order_id
        
    def change_order_status(self, order_status):
        self.order_status = order_status

    def set_filled_qty(self, filled_qty):
        self.filled_qty = filled_qty

    def set_pending_qty(self, pending_qty):
        self.pending_qty = pending_qty

    def set_order_timestamp(self, order_timestamp):
        self.order_timestamp = order_timestamp

    def set_exchange_timestamp(self, exchange_timestamp):
        self.exchange_timestamp = exchange_timestamp

    def set_message(self, message):
        self.message = message

In [13]:
def placeOrder(order):
    broker = "Zerodha"
    kite = kite_handler
    try:
        order_id = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=order.exchange,
        tradingsymbol=order.trading_symbol,
        transaction_type=order.transaction_type,
        quantity=order.quantity,
        price=order.price,
        trigger_price=order.trigger_price,
        product=order.product,
        order_type=order.order_type)
        logging.info(f'Order placed successfully, order_id = {order_id}')
        order.set_order_id(order_id)
    except Exception as e:
        logging.info("Order placement failed: {e}")
        raise Exception(str(e))

In [18]:
try_order = Order(variety=KiteConstants.VARIETY_REGULAR, exchange=KiteConstants.EXCHANGE_NSE, trading_symbol="ITC", transaction_type=KiteConstants.TRANSACTION_TYPE_BUY, quantity=1, product=KiteConstants.PRODUCT_MIS, order_type=KiteConstants.ORDER_TYPE_MARKET)

In [18]:
try_order_2 = Order(variety=KiteConstants.VARIETY_REGULAR, exchange=KiteConstants.EXCHANGE_NSE, trading_symbol="ITC", transaction_type=KiteConstants.TRANSACTION_TYPE_BUY, quantity=1, product=KiteConstants.PRODUCT_MIS, order_type=KiteConstants.ORDER_TYPE_MARKET)

In [19]:
placeOrder(try_order)

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /orders/regular HTTP/11" 200 None
INFO:root:Order placed successfully, order_id = 241219201577784


In [19]:
placeOrder(try_order_2)

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /orders/regular HTTP/11" 200 None
INFO:root:Order placed successfully, order_id = 241219201546167


In [20]:
try_order_2.order_id

'241219201546167'

In [23]:
data = {'account_id': 'BI3034', 'unfilled_quantity': 0, 'checksum': '', 'placed_by': 'BI3034', 'order_id': '241219201546167', 'exchange_order_id': '1100000042846003', 'parent_order_id': None, 'status': 'COMPLETE', 'status_message': None, 'status_message_raw': None, 'order_timestamp': '2024-12-19 13:37:24', 'exchange_update_timestamp': '2024-12-19 13:37:24', 'exchange_timestamp': '2024-12-19 13:37:24', 'variety': 'regular', 'exchange': 'NSE', 'tradingsymbol': 'ITC', 'instrument_token': 424961, 'order_type': 'MARKET', 'transaction_type': 'BUY', 'validity': 'DAY', 'product': 'MIS', 'quantity': 1, 'disclosed_quantity': 0, 'price': 0, 'trigger_price': 0, 'average_price': 467, 'filled_quantity': 1, 'pending_quantity': 0, 'cancelled_quantity': 0, 'market_protection': 0, 'meta': {}, 'tag': None, 'guid': '130032XxOepiy817I2E'}
pprint(data)

{'account_id': 'BI3034',
 'average_price': 467,
 'cancelled_quantity': 0,
 'checksum': '',
 'disclosed_quantity': 0,
 'exchange': 'NSE',
 'exchange_order_id': '1100000042846003',
 'exchange_timestamp': '2024-12-19 13:37:24',
 'exchange_update_timestamp': '2024-12-19 13:37:24',
 'filled_quantity': 1,
 'guid': '130032XxOepiy817I2E',
 'instrument_token': 424961,
 'market_protection': 0,
 'meta': {},
 'order_id': '241219201546167',
 'order_timestamp': '2024-12-19 13:37:24',
 'order_type': 'MARKET',
 'parent_order_id': None,
 'pending_quantity': 0,
 'placed_by': 'BI3034',
 'price': 0,
 'product': 'MIS',
 'quantity': 1,
 'status': 'COMPLETE',
 'status_message': None,
 'status_message_raw': None,
 'tag': None,
 'tradingsymbol': 'ITC',
 'transaction_type': 'BUY',
 'trigger_price': 0,
 'unfilled_quantity': 0,
 'validity': 'DAY',
 'variety': 'regular'}


In [16]:
def order_update(ws, data):
    logging.info(data)
    if(try_order.order_id == data['order_id']):
        try_order.set_exchange_timestamp(data['exchange_timestamp'])
        try_order.set_filled_qty(data['filled_quantity'])
        try_order.set_pending_qty(data['pending_quantity'])
        try_order.change_order_status(data['status'])

In [17]:
kws.on_order_update = order_update

In [25]:
try_order.order_status

'COMPLETE'